In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import glob
import numpy as np
from matplotlib.colors import ListedColormap

# Compare iHS and Tajima on continents

In [4]:
population_groups = {
    "AFR": ["ACB", "ASW", "ESN", "GWD", "LWK", "MSL", "YRI"],
    "AMR": ["CLM", "MXL", "PEL", "PUR"],
    "EAS": ["CDX", "CHB", "CHS", "JPT", "KHV"],
    "EUR": ["CEU", "FIN", "GBR", "IBS", "TSI"],
    "SAS": ["BEB", "GIH", "ITU", "PJL", "STU"]
}

In [5]:
rs_ids = pd.read_csv('../data/snp_coordinates_grch37.tsv', sep='\t')
iHS_scored = pd.DataFrame(columns=['continent', 'country', 'iHS_score', 'chromosome', 'rsID'])

rows = []
for continent, populations in population_groups.items():
    for pop in populations:
        for _, row in rs_ids.iterrows():
            rsID = row['rsID']
            chromosome = row['Chromosome']
            
            path = f"/Users/emilevancauwenberghe/Documents/LargeFiles/Database/ihs_hapbin_globalCM/{pop}/hapbin_globalCM/{pop}.chr{chromosome}.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.noCNV.snv.modAncDer.bcfCheck.hapbin.ihs.txt"
            ihs_data = pd.read_csv(path, sep='\t')
            match = ihs_data.loc[ihs_data['Location'] == rsID, 'iHS'].values
            iHS_score = match[0] if len(match) > 0 else None
            rows.append({
                'continent': continent,
                'country': pop,
                'iHS_score': iHS_score,
                'chromosome': chromosome,
                'rsID': rsID
            })
iHS_scored = pd.DataFrame(rows)

## Grid plot of the iHS scores

In [40]:
pivot = iHS_scored.pivot(index='country', columns='rsID', values='iHS_score')

base_cmap = sns.color_palette("vlag", as_cmap=True)
cmap = ListedColormap(base_cmap(np.linspace(0, 1, 256)))
cmap.set_bad(color='gray')

fig_width = max(10, pivot.shape[1] * 0.4)
fig_height = max(6, pivot.shape[0] * 0.4)
plt.figure(figsize=(fig_width, fig_height))

ax = sns.heatmap(pivot, cmap=cmap, cbar=True, linewidths=0.5, linecolor='gray')

ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

plt.xlabel('rsID')
plt.ylabel('Population')
plt.title('iHS Score par Population et rsID')
plt.tight_layout()

plt.savefig("../results/ihs_heatmap.png", dpi=300, bbox_inches='tight')
plt.close()

In [46]:
iHS_scored = iHS_scored.groupby(['rsID', 'continent']).agg(
    nan_count = ('iHS_mean', lambda x: x.isna().sum()),
    iHS_std = ('iHS_mean', 'std'),
    iHS_mean = ('iHS_mean', 'mean')
).reset_index()
iHS_scored

,rsID,continent,nan_count,iHS_std,iHS_mean
0,rs10875612,AFR,0,NaN,-0.838705
1,rs10875612,AMR,0,NaN,-1.248393
2,rs10875612,EAS,0,NaN,-0.551481
3,rs10875612,EUR,0,NaN,-0.801984
4,rs10875612,SAS,0,NaN,-0.110292
...,...,...,...,...,...
130,rs9969232,AFR,0,NaN,-4.337790
131,rs9969232,AMR,0,NaN,-1.635387
132,rs9969232,EAS,0,NaN,-2.718750
133,rs9969232,EUR,0,NaN,-0.572404


In [47]:
Tajima_scores = pd.read_csv("../results/tajimas_d_results.csv",header=1)
Tajima_scores = Tajima_scores.rename(columns={Tajima_scores.columns[0]: 'rsID'})
Tajima_scores = Tajima_scores.melt(id_vars='rsID', var_name='continent', value_name='Tajima_score')

In [48]:
sorted(Tajima_scores["rsID"].unique()) == sorted(iHS_scored["rsID"].unique())

True

In [49]:
merged_results = pd.merge(iHS_scored, Tajima_scores, on=['rsID', 'continent'], how='inner')
merged_results
merged_results.to_csv('../results/merged_results.csv', index=False)

In [50]:
sns.scatterplot(data=merged_results, x='iHS_mean', y='Tajima_score', hue='continent')
plt.xlabel('iHS Score')
plt.ylabel('Tajima’s D')
plt.title('Comparison of iHS and Tajima’s D scores')
plt.savefig("../results/comparison_iHS_Tajima_D.png", dpi=300, bbox_inches='tight')
plt.close()


clean_df = merged_results[['iHS_mean', 'Tajima_score']].dropna()
corr = clean_df.corr(method='spearman')
print(corr)

              iHS_mean  Tajima_score
iHS_mean      1.000000      0.004269
Tajima_score  0.004269      1.000000
